In [15]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [16]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [17]:
include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [18]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1473,1.8543,0.1012,0.4365,-2.9008,-0.6469,-0.1883,0.0258,2.1941,0.6874,0.1474
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1479,1.7607,0.0658,0.4053,-2.9112,-0.596,-0.2186,0.032,1.9119,0.445,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0980477
 0.0980477  1.0

In [19]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

scenarios = block_bootstrap(values(data), 10_000, block_size=60, random_block=true)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0615,0.0246
5,0.0619,0.0232
10,0.062,0.0225
25,0.0623,0.0222


,Total_Stock_CPI,Price_10Y_CPI
1,0.1559,0.0878
5,0.0679,0.0415
10,0.0478,0.0309
25,0.0297,0.0203


,Total_Stock_CPI,Price_10Y_CPI
1,-0.761,0.4653
5,-0.2362,0.2649
10,-0.3058,0.1072
25,-0.1843,0.1272


,Total_Stock_CPI,Price_10Y_CPI
1,1.0074,1.0259
5,-0.2452,0.8609
10,-0.329,-0.0195
25,-0.1977,-0.1528


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4418,-0.2905,-0.2182,-0.0233,0.0818,0.1636,0.2818,0.3138,0.3615
5,-0.0942,-0.074,-0.0508,0.0119,0.0708,0.1095,0.1676,0.19,0.2104
10,-0.0506,-0.0335,-0.0226,0.0302,0.0653,0.0976,0.1329,0.1436,0.1585
25,-0.0086,0.0015,0.0119,0.0421,0.0634,0.0834,0.1093,0.1168,0.1244


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1745,-0.1293,-0.1031,-0.0341,0.0203,0.0786,0.1699,0.2267,0.2852
5,-0.0752,-0.0616,-0.0431,-0.0031,0.0214,0.0495,0.0916,0.1131,0.1443
10,-0.0466,-0.0357,-0.0262,0.0013,0.0222,0.0428,0.0748,0.086,0.0947
25,-0.0217,-0.0158,-0.0102,0.0077,0.0219,0.0361,0.056,0.0623,0.0699


In [20]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.421327  0.521378  0.539183  0.592982  0.724743

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 60.0  106.0  145.0  191.0  352.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.199936  0.376498  0.430734  0.498436  0.650713

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 78.0  155.0  225.0  332.0  653.03